# YOLACT Google Colab Notebook

Yes, ini modifikasi Aria.

Inference on images and video with [YOLACT](https://github.com/dbolya/yolact).

## Google Colab
This notebook is set up to work inside Google Colab, which is a free, Linux-based Jupyter Notebook environment hosted in the cloud.

If you try it outside of Google Colab I'm not sure if it will work. I originally had problems running locally on my Windows machine, which is why I switched to this.

### Runtime Setup
- Go to Runtime > Change Runtime Type
- Choose GPU (TPU won't work)

## YOLACT
YOLACT is a state of the art, real-time, single shot object segmentation algorithm detailed in these papers:  
- [YOLACT: Real-time Instance Segmentation](https://arxiv.org/abs/1904.02689)
- [YOLACT++: Better Real-time Instance Segmentation](https://arxiv.org/abs/1912.06218)

**Big thanks to the authors: Daniel Bolya, Chong Zhou, Fanyi Xiao, Yong Jae Lee!**

## Immersive Limit
This notebook was created by Adam Kelly, instructor/creator of [immersivelimit.com](https://www.immersivelimit.com).
- Subscribe for lots more awesome AI and 3D content on the [YouTube channel](https://www.youtube.com/c/immersivelimit)
- Check out https://www.immersivelimit.com/connect for more ways to follow/connect 😀

# Initial Setup
First, we install some Python libraries. Google Colab probably has these requirements already, but better to include them than to break later.

In [1]:
# Cython needs to be installed before pycocotools
!pip install cython
!pip install opencv-python pillow pycocotools matplotlib

In [2]:
# Downgrade torch to accommodate DCNv2
!pip install torchvision==0.5.0
!pip install torch==1.4.0


     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 753.4 MB 7.1 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu111
    Uninstalling torchvision-0.10.0+cu111:
      Successfully uninstalled torchvision-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.4.0 which is incompatible.


## Clone YOLACT from GitHub
Let's get that github repo! If anything in this notebook breaks, check the repo in case they've made updates.

In [3]:
# Make sure we're in the top folder
%cd /content

# Clone the repo
!git clone https://github.com/ariaamadeus/yolact.git

/content
Cloning into 'yolact'...
remote: Enumerating objects: 2953, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 2953 (delta 5), reused 0 (delta 0), pack-reused 2937
Receiving objects: 100% (2953/2953), 21.23 MiB | 23.91 MiB/s, done.
Resolving deltas: 100% (2008/2008), done.


## DCNv2
The DCNv2 external library is needed for this to work, so we'll build it now.

**IF YOU GET CUDA ERRORS** You need to change the Runtime of this notebook to "GPU" as mentioned at the top of the notebook.

In [4]:
# Change to the right directory
%cd /content/yolact/external/DCNv2

# Build DCNv2
!python setup.py build develop

/content/yolact/external/DCNv2
running build
running build_ext
building '_ext' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/content
creating build/temp.linux-x86_64-3.7/content/yolact
creating build/temp.linux-x86_64-3.7/content/yolact/external
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src/cpu
creating build/temp.linux-x86_64-3.7/content/yolact/external/DCNv2/src/cuda
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -DWITH_CUDA -I/content/yolact/exte

## Pretrained Weights
In order to run inference, we'll need some pre-trained weights. The creator of the GitHub repo shared them on Google Drive. We're going to use a [helpful tool](https://github.com/chentinghao/download_google_drive) made by [chentinghao](https://github.com/chentinghao) to easily access the Drive file from Colab.

If this stops working, it's probably because the weights moved to a different location. Check the YOLACT github repo to see where they went.

In [5]:
# Make sure we're in the top folder
%cd /content

# Clone the repo
!git clone https://github.com/chentinghao/download_google_drive.git

# Create a new directory for the pre-trained weights
!mkdir -p /content/yolact/weights

# Download the file
!python ./download_google_drive/download_gdrive.py 1Jy3yCdbatgXa5YYIdTCRrSV0S9V5g1rn ./yolact/weights/resnet50-19c8e357.pth

/content
Cloning into 'download_google_drive'...
remote: Enumerating objects: 16, done.
remote: Total 16 (delta 0), reused 0 (delta 0), pack-reused 16
Unpacking objects: 100% (16/16), done.
97.8MB [00:01, 100MB/s]


# Get Test Images
We'll download a few test images from the [COCO dataset](http://cocodataset.org/#explore). Feel free to try out your own images as well, but know that there are only 80 categories and they're not intended to cover everything, so if you upload a picture of a snake, a mouse, and a cat, only the cat will be detected.

In [ ]:
# Make a new directory for the test images
!mkdir /content/test_images

# Move to the new directory
%cd /content/test_images

# Make a list of URLs from the COCO dataset
get_imgs = ['http://farm8.staticflickr.com/7200/6946559805_7edf757660_z.jpg',
            'http://farm3.staticflickr.com/2835/9520716824_51220b13d0_z.jpg',
            'http://farm3.staticflickr.com/2817/10105052924_0c8d79e62c_z.jpg',
            'https://farm3.staticflickr.com/2637/3679968895_37a79e55b3_z.jpg',
            'https://farm8.staticflickr.com/7137/7705349298_6bea29cf1c_z.jpg',
            'http://farm5.staticflickr.com/4105/4981945644_3f162314ed_z.jpg',
            'http://farm1.staticflickr.com/40/101757885_a8fc40e273_z.jpg',
            'http://farm3.staticflickr.com/2654/3897535041_fa8dd5570b_z.jpg']

# Download all the images
for img_url in get_imgs:
  !wget {img_url} -nc

# List the contents of the folder. Should have images.
!pwd
!ls

# Run Inference on Images
Let's run inference. We do that with eval.py

In [19]:
# Move up to the top level directory
%cd /content/yolact

# Delete any previous output_images folder (in case we're running this more than once)
!rm /content/yolact/output_images -r

# Create a new directory for output images
!mkdir -p /content/yolact/output_images

# Run inference using our pre-trained weights on all images in the directory
!python eval.py --trained_model=weights/yolact_plus_resnet50_kelapa_sawit_77_544_interrupt.pth --config=yolact_resnet50_kelapa_sawit_config --score_threshold=0.15 --top_k=15 --images=KelapaTest:output_images

/content/yolact
Loading model... Done.

KelapaTest/REJECT_(68).JPG -> output_images/REJECT_(68).png
KelapaTest/REJECT_(67).JPG -> output_images/REJECT_(67).png
KelapaTest/REJECT_(65).JPG -> output_images/REJECT_(65).png
KelapaTest/ACCEPT_(79).JPG -> output_images/ACCEPT_(79).png
KelapaTest/ACCEPT_(78).JPG -> output_images/ACCEPT_(78).png
KelapaTest/ACCEPT_(88).JPG -> output_images/ACCEPT_(88).png
KelapaTest/ACCEPT_(89).JPG -> output_images/ACCEPT_(89).png
KelapaTest/ACCEPT_(73).JPG -> output_images/ACCEPT_(73).png
KelapaTest/REJECT_(66).JPG -> output_images/REJECT_(66).png
KelapaTest/REJECT_(63).JPG -> output_images/REJECT_(63).png
Done.


## Display Output Images
The last step saved output images, so we'll display those now.

In [22]:
%cd /content/yolact

import cv2
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

output_images = Path('output_images')

def show_image(img_path):
  img = cv2.imread(img_path)
  img_cvt=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  plt.figure(figsize=(16,16))
  plt.imshow(img_cvt)
  plt.show()

# Iterate through all of the output images and display them
for img_path in output_images.iterdir():
  print(img_path)
  show_image(str(img_path))

Output hidden; open in https://colab.research.google.com to view.

# Run Inference on Video
I can't figure out how to get Google Colab to show the output video in the notebook, so I've uploaded the video to YouTube. The mask quality is incredible. There are some false positives because it thinks the rocks are sheep, but we'll give it a pass because it's never been told what a rock is before (COCO dataset doesn't contain rocks) and the rocks do look a bit sheepish.


In [ ]:
from IPython.display import HTML
HTML('<iframe width="1280" height="720" src="https://www.youtube.com/embed/bQgtmbzN7jg" frameborder="0" allow="accelerometer; autoplay; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>')

If you'd like to process your own video, you can modify the steps below.

In [ ]:
# Make a new directory for the test images
!mkdir -p /content/test_video

# Move to the top level directory
%cd /content

# Download the file
# https://drive.google.com/file/d/1I4fivRLniVNt_LcWqhQNehTSdEkZ4ytW/view?usp=sharing
# Note that file_id is extracted from the share URL above, it'll be different for every file
file_id = "1I4fivRLniVNt_LcWqhQNehTSdEkZ4ytW"
file_path = "/content/test_video/giraffes_full_720.mp4" # <- this is the destination path

!python /content/download_google_drive/download_gdrive.py {file_id} {file_path}

!ls /content/test_video

Now we run inference.

In [ ]:
%cd /content
!mkdir -p /content/output_videos
output_path = "/content/output_videos/giraffes_full_720_output.mp4"
!python ./yolact/eval.py --trained_model=./yolact/weights/yolact_plus_resnet50_54_800000.pth --score_threshold=0.15 --top_k=15 --video_multiframe=4 --video={file_path}:{output_path}

## Download the output file
In this part, we'll mount your Google Drive and you can copy the file somewhere over there. I tried using the google.colab.files library to download the file, but it doesn't seem to like big video files.

- You will need to authorize your Drive to connect, then it basically just links your drive as if it's a folder on this cloud computer.
- **Be careful** because if you start deleting/modifying files in this folder, you will modify your personal Google Drive.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p /content/drive/My\ Drive/YOLACT_output
%cd /content/drive/My\ Drive/YOLACT_output
!ls

In [ ]:
!cp {output_path} /content/drive/My\ Drive/YOLACT_output/giraffes_full_720_output.mp4

## Find the Video on Google Drive
Now you should be able to view the file on your Google Drive in the YOLACT_output folder.

# Conclusion
🎉 Woohoo! You did it! 🎉

Thanks for checking out the notebook, I hope it was helpful!

**If you liked it, consider sharing it!** YOLACT is pretty awesome, so more people should know about it.


### Want to learn how to Train on a custom dataset?

Check out this tutorial:

[Train YOLACT with a Custom COCO Dataset | Immersive Limit](https://www.immersivelimit.com/tutorials/train-yolact-with-a-custom-coco-dataset)

Install library lagi

In [6]:
!pip install wandb -qqq
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 139 kB 43.3 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 180 kB 46.6 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
  Cloning https://github.com/philferriere/cocoapi.git to /tmp/pip-req-build-5qpylmf8
  Running command git clone -q https://github.com/philferriere/cocoapi.git /tmp/pip-req-build-5qpylmf8
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=264149 sha256=b509793813767aec39edb298ac2290d4e009ecee293d233a72b23da880d18a1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-52j11f3h/wheels/6b/c6/c5/cb6da4cb793a6cb1ab91f6578d76c42686422127eb4dbcea94
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.2
    Uninstalling pycocotools-2.0.2:
      Successfully uninstalled pycocotools-2.0.2


Download dataset:

In [7]:
!python ./download_google_drive/download_gdrive.py 1p6jucgJufNO_67J4OwvZHAb4__YtAdOL ./yolact/data/KelapaSawit.zip

26.0MB [00:00, 48.1MB/s]


Unzip/Unrar

In [8]:
%cd /content/yolact/data
!unrar x KelapaSawit.rar
!unzip KelapaSawit.zip

/content/yolact/data

UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal

Cannot open KelapaSawit.rar
No such file or directory
No files to extract
Archive:  KelapaSawit.zip
   creating: KelapaSawit/
   creating: KelapaSawit/Kelapa Test/
  inflating: KelapaSawit/Kelapa Test/ACCEPT_(73).JPG  
  inflating: KelapaSawit/Kelapa Test/ACCEPT_(78).JPG  
  inflating: KelapaSawit/Kelapa Test/ACCEPT_(79).JPG  
  inflating: KelapaSawit/Kelapa Test/ACCEPT_(88).JPG  
  inflating: KelapaSawit/Kelapa Test/ACCEPT_(89).JPG  
  inflating: KelapaSawit/Kelapa Test/REJECT_(63).JPG  
  inflating: KelapaSawit/Kelapa Test/REJECT_(65).JPG  
  inflating: KelapaSawit/Kelapa Test/REJECT_(66).JPG  
  inflating: KelapaSawit/Kelapa Test/REJECT_(67).JPG  
  inflating: KelapaSawit/Kelapa Test/REJECT_(68).JPG  
   creating: KelapaSawit/KelapaSet/
   creating: KelapaSawit/KelapaSet/ImageKelapa/
  inflating: KelapaSawit/KelapaSet/ImageKelapa/000000.JPG  
  inflating: KelapaSawit/KelapaSet/ImageKelapa/000001

In [9]:
%cd /content/yolact
!python ./train.py --config=yolact_resnet50_kelapa_sawit_config

/content/yolact
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 2
wandb: You chose 'Use an existing W&B account'
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Initializing weights...
Begin training!

wandb: Currently logged in as: multimeter (use `wandb login --relogin` to force relogin)
wandb: Tracking run with wandb version 0.12.4
wandb: Syncing run hopeful-djinn-39
wandb: ⭐️ View project at https://wandb.ai/multimeter/TesKelapaSawit
wandb: 🚀 View run at https://wandb.ai/multimeter/TesKelapaSawit/runs/2q51jrnc
wandb: Run data is saved locally in /content/yolact/wandb/run-20